In [ ]:
# suatu sistem rekomendasi dapat dikatakan memiliki akurasi yang tinggi apabila semakin banyak GT item yang dapat dengan tepat direkomendasikan pada rangking teratas

In [1]:
# import 

import pandas as pd 
import numpy as np 
import math as math
import joblib
from joblib import dump
import os
import time

## PRECISION

$$ Precision(Gt_u, TopN_u, N) = \frac{\left| TopN_u \cap GT_u \right|}{\left| TopN_u(N) \right|} $$
$$ TopN_u(N) = N, sehingga $$
$$ Precision(Gt_u, TopN_u, N) = \frac{\left| TopN_u \cap GT_u \right|}{\left| N \right|} $$

In [82]:
# toy data 
gt_user1 = [1, 2, 3, 4, 11, 12]  # ground truth relevan
topn_user1 = [1, 5, 2, 3, 15, 4]  # hasil rekomendasi sistem


def Precision(GroundTruth, TopN, N):
    rumusPrecision = len(np.intersect1d(TopN[:N], GroundTruth)) / N
    print(len(np.intersect1d(TopN[:N], GroundTruth)))
    # print(rumusPrecision)
    return rumusPrecision

In [86]:
calPrecsionMatrik = Precision(gt_user1, topn_user1, 3)
print(calPrecsionMatrik)

2
0.6666666666666666


## RECALL


$$ Recall(GT_u, TopN_u, N) = \frac{\left| TopN_u \cap GT_u \right|}{\left| GT(N) \right|} $$

In [25]:

def Recall(GT, TopN, N):
    rumusRecall = len(np.intersect1d(TopN[:N], GT)) / len(set(GT))
    print(rumusRecall)
    return(rumusRecall)

In [26]:
calRecalMatriks = Recall(gt_user1, topn_user1, 4)

0.6


## F1 SCORE

$$ f1Score(GT_u, TopN_u, N) = \frac{\left| 2 * Precision(Gt_u, TopN_u, N) * Recall(Gt_u, TopN_u, N)  \right|}{\left| Precision(Gt_u, TopN_u, N) + Recall(Gt_u, TopN_u, N)  \right|} $$

In [27]:
def F1Score(GT, TopN, N):
    precision = Precision(GT, TopN, N)
    recall = Recall(GT, TopN, N)
    rumusF1Score = ((2 * precision * recall) / (precision + recall)) 
    if precision > 0 and recall > 0: 
        rumusF1Score = rumusF1Score
    else :
        rumusF1Score = 0
    print(rumusF1Score)
    return rumusF1Score
    

In [28]:
calF1Score = F1Score(gt_user1, topn_user1, 4)

0.6
0.6666666666666665


## AP(Average Precision)


$$
AP(GT_u, TopN_u, N) = \frac{1}{|GT_u|} \sum_{n=1}^{N} Precision(GT_u, TopN_u, N)  \cdot \parallel (TopN_u(n) \in  GT_u) $$



In [29]:
def AP(GT, TopN, N):
    aPrecision = np.array([Precision(GT, TopN, N=x) for x in range(1, N+1)])
    bGroundTruth = np.array([np.sum(np.where(tp == GT, 1, 0)) for tp in TopN[:N]])
    rumusAP = (1/len(GT) * np.sum(aPrecision * bGroundTruth))
    print(rumusAP)
    return rumusAP

In [30]:
calAPMatriks = AP(gt_user1, topn_user1, 4)

0.0


## MSE

$$ MSE = \frac{\sum_{(u,i)\in GT^{e^{2}}ui}^{}}{\left| GT \right|} $$

$$ dimana, e = r_{ui} - \hat{r_{ui}} $$

In [52]:
## toy data
gt = [2, 4, 6, 3]
pred = [2.5, 5.4, 4.3, 1]


In [46]:
def MSE(groundTruth, ratingPrediksi):
    # Convert lists to NumPy arrays
    groundTruth_np = np.array(groundTruth)
    ratingPrediksi_np = np.array(ratingPrediksi)
    
    # Calculate MSE correctly: mean of the squared differences
    rumusMSE = np.mean((groundTruth_np - ratingPrediksi_np) ** 2)
    return rumusMSE

In [55]:
calMSE = MSE(gt, pred)
print(calMSE)

2.2750000000000004


## RMSE

$$ RMSE = \sqrt{\frac{\sum_{(u,i)\in GT^{e^{2}}ui}^{}}{\left| GT \right|}} $$


$$ dimana, e = r_{ui} - \hat{r_{ui}} $$

In [59]:
def RMSE (groundTruth, ratingPrediksi):
    rumusRMSE = np.sqrt(MSE(groundTruth, ratingPrediksi))
    return rumusRMSE

In [60]:
calMSE = RMSE(gt, pred)
print(calMSE)

1.5083103128998356


## MAE


$$ MAE = \frac{\sum_{(u,i) \in  E}^{}\left| e_{ui} \right|}{\left| E \right|} $$

In [61]:
def MAE (groundTruth, ratingPrediksi):
    groundTruth_np = np.array(groundTruth)
    ratingPrediksi_np = np.array(ratingPrediksi)
    rumusMAE = np.mean(np.abs(groundTruth_np - ratingPrediksi_np))
    return rumusMAE

In [63]:
calMAE = MAE(gt, pred)
print(calMAE)

1.4000000000000001


## NRMSE

$$ NRMSE = \frac {RMSE}{r_{max} - r_{min}} $$

In [66]:
def NRMSE(groundTruth, ratingPrediksi):
    groundTruth_np = np.array(groundTruth)
    ratingPrediksi_np = np.array(ratingPrediksi)
    rumusNRMSE = RMSE(groundTruth_np, ratingPrediksi_np) / (groundTruth_np.max() - groundTruth_np.min())
    return rumusNRMSE

In [67]:
calNRMSE = NRMSE(gt, pred)
print(calNRMSE)

0.3770775782249589


## NMAE

$$ NMAE = \frac {MAE}{r_{max} - r_{min}} $$ 

In [68]:
def NMAE(groundTruth, ratingPrediksi):
    groundTruth_np = np.array(groundTruth)
    ratingPrediksi_np = np.array(ratingPrediksi)
    rumusNMAE = MAE(groundTruth_np, ratingPrediksi_np) / (groundTruth_np.max() - groundTruth_np.min())
    return rumusNMAE

In [69]:
calNMAE = NMAE(gt, pred)
print(calNMAE)

0.35000000000000003
